In [1]:
import bhnerf
from astropy import units
import jax
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from bhnerf import constants as consts

Welcome to eht-imaging! v 1.2.2 



2023-07-12 12:18:12.701102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [145]:
def make_ehtim_movie(image_plane, t_array, fov_M, stokes, mjd=57854, ra=12.513729999999999, dec=12.39112):
    import ehtim as eh
    fov_rad = (fov_M * consts.GM_c2(consts.sgra_mass) / consts.sgra_distance.to('m')) * units.rad
    npix = image_plane.shape[-1]
    im_list = []
    for t, image in zip(t_array, image_plane):
        im = eh.image.make_empty(npix, fov_rad.value, ra, dec, mjd=mjd, time=t.value)
        for i, s in enumerate(stokes):
            im.add_pol_image(np.flipud(scipy.ndimage.rotate(image[i], -158-90, reshape=False)), s)
        im_list.append(im)
    movie = eh.movie.merge_im_list(im_list)
    return movie

Generate synthetic ALMA lightcurves 
---
Polarized lightcurves corresponding to ALMA observation times on April 11, 2017. \
Generate synthetic hot-spot data and the resulting polarized image-plane.

In [6]:
"""
Load ALMA Data and time average over a window
"""
data_path = '../data/Apr11_HI.dat'
alma_lc = pd.read_csv(data_path)

t0 = 9. + 20./60.           # UTC
t7 = t0 + 68./60. + 35./60. # UTC

window_size = 8
alma_lc_loops = alma_lc.loc[np.bitwise_and(alma_lc['time']>=t0, alma_lc['time']<=t7)]
alma_lc_means = alma_lc_loops.rolling(window_size).mean().loc[::window_size].dropna()
t_frames = alma_lc_means['time'].values * units.hr

%matplotlib widget
fig, axes = plt.subplots(1, 4, figsize=(10,2.7))
for s, ax in zip(['I', 'Q', 'U', 'V'], axes):
    ax.errorbar(alma_lc.time,  alma_lc[s], fmt='.', ecolor='black')
    ax.errorbar(alma_lc_means.time,  alma_lc_means[s], fmt='.', color='tab:orange')
    ymin = alma_lc[s].min() - 0.2*np.abs(alma_lc[s].min())
    ymax = alma_lc[s].max() + 0.2*np.abs(alma_lc[s].max())
    ax.fill_between([t0, t7], [ymax, ymax], ymin, alpha=0.3, color='gray')
    ax.set_ylim([ymin, ymax])
    ax.set_title('{} lightcurve'.format(s))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
stokes = ['Q', 'U']
target = np.array(alma_lc_means[stokes])

t_model = np.linspace(9.2, 11.15, 100) * units.hr
rot_angle = np.deg2rad(120)
inclination = np.deg2rad(21.0)

orbit_radius = 11.0
spin = 0.0

Omega_dir = 'cw'
rot_sign = {'cw': -1, 'ccw': 1}

b_consts = [0, 1, 0]
Q_frac = 0.85
V_frac = 0

I_sha = 2.4  # Jy
P_sha = 0.16 # Jy
chi_sha = np.deg2rad(-37)
qu_sha = P_sha * np.array([np.cos(2*chi_sha), np.sin(2*chi_sha)])
iqu_sha = np.concatenate(([I_sha], qu_sha))

de_rot_data = np.deg2rad(32.2)
de_rot_model = np.deg2rad(20.0)

%matplotlib widget
plt.figure(figsize=(2.5,2.5))
plt.scatter(target[:, 0], target[:, 1], c=alma_lc_means['time'], cmap=plt.cm.get_cmap('inferno'), s=1)
plt.title('Q-U Loop');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Orbiting point source
--
A semi-analytic model ([Gelles et al.](https://arxiv.org/abs/2105.09440)) used in [Weilgus et al.](https://www.aanda.org/articles/aa/full_html/2022/09/aa44493-22/aa44493-22.html) to analyze polarimetric ALMA observations

In [30]:
"""
This code re-creates the resulting using kgeo
"""
ngeo = 500
mbar = 0

flux_scale = 0.01

kepler_period = 2 * np.pi * (orbit_radius**(3/2) + spin) * consts.GM_c3(consts.sgra_mass.to('kg')).to('hr')
varphis = rot_sign[Omega_dir] * (2 * np.pi * (t_model - t_frames[0]) / kepler_period).value + rot_angle

observer_coords = [0, 1000.0, inclination, 0]
alpha, beta = bhnerf.kgeo.equatorial_lensing.rho_of_req(spin, inclination, req=orbit_radius, mbar=mbar, varphis=varphis)[-2:]
geos = bhnerf.kgeo.raytrace_ana(spin, observer_coords, [alpha, beta], ngeo, plotdata=False, verbose=False).get_dataset()
emission = bhnerf.emission.equatorial_ring(geos, mbar)

Omega = rot_sign[Omega_dir] * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))
umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)

b = bhnerf.kgeo.magnetic_field_spherical(geos, *b_consts)
J = emission.data * bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac, V_frac)
iqu_semi_analytic = bhnerf.kgeo.radiative_trasfer(J, g, 1.0, np.array(geos.Sigma)).T
qu_semi_analytic = flux_scale*bhnerf.emission.rotate_evpa(iqu_semi_analytic[:, 1:], de_rot_model, axis=1) + qu_sha

# Intrinsic curved (de-rotated)
alma_qu_intrinsic = bhnerf.emission.rotate_evpa(target-qu_sha, de_rot_data, axis=1)
qu_semi_analytic_intrinsic = flux_scale*bhnerf.emission.rotate_evpa(iqu_semi_analytic[:, 1:], de_rot_model + de_rot_data, axis=1)

In [45]:
%matplotlib widget
fig, axes = plt.subplots(2, 3, figsize=(8,6))
bhnerf.visualization.plot_stokes_lc(target, stokes, t_frames, label='True', plot_qu=True, axes=axes[0])
bhnerf.visualization.plot_stokes_lc(qu_semi_analytic, stokes, t_model, label='Point source', plot_qu=True, axes=axes[0], fmt='--', color='r')

# Intrinsic plots
bhnerf.visualization.plot_stokes_lc(alma_qu_intrinsic, stokes, t_frames, label='True', plot_qu=True, axes=axes[1])
bhnerf.visualization.plot_stokes_lc(qu_semi_analytic_intrinsic, stokes, t_model, label='Point source', plot_qu=True, axes=axes[1], fmt='--', color='r')

for ax in axes.ravel(): ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Gaussian hotspot
---
A gaussian hotspot in a keplerian orbit without shear similar to [Weilgus et al.](https://www.aanda.org/articles/aa/full_html/2022/09/aa44493-22/aa44493-22.html)

In [46]:
hs_fwhm = 6
hs_std = hs_fwhm / 2.355
fov_M = 40.0 
rmax = fov_M / 2
GM_c3 = consts.GM_c3(consts.sgra_mass).to(t_frames.unit)

t_model = np.linspace(t_frames[0].value, 11.15, 100) * units.hr

delta_t_M = (t_frames[0]-t_model[0]) / GM_c3

# Keplerian prograde velocity field
Omega_rs = rot_sign[Omega_dir] / (orbit_radius**(3/2) + spin)

# Generate hotspot measurements (see Tutorial2)
gaussian_hs = bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=rot_angle + np.pi/2 - float(Omega_rs * delta_t_M),
    orbit_radius=orbit_radius,
    std=hs_std,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2'))

geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=128, num_beta=128, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2])
geos.dtau[:] = 1.0
Omega = Omega_rs

umu = bhnerf.kgeo.azimuthal_velocity_vector(geos, Omega)
g = bhnerf.kgeo.doppler_factor(geos, umu)
b = bhnerf.kgeo.magnetic_field_spherical(geos, *b_consts)

J = np.nan_to_num(bhnerf.kgeo.parallel_transport(geos, umu, g, b, Q_frac=Q_frac, V_frac=V_frac), 0.0)
J_inds = [['I', 'Q', 'U'].index(s) for s in stokes]
J_rot = bhnerf.emission.rotate_evpa(J[J_inds], np.deg2rad(20))

/scratch/imaging/projects/bh_nerf/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:700: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
/scratch/imaging/projects/bh_nerf/bhNeRF/bhnerf/kgeo.py:494: RuntimeWarning: invalid value encountered in sqrt
  cot_th_b = np.sqrt(1 - sin_th_b**2) / sin_th_b


In [57]:
flux_scale = 0.004
image_plane_hs = bhnerf.emission.image_plane_dynamics(
    gaussian_hs, geos, Omega, t_model, t_injection=0, J=J_rot, slow_light=False)
qu_hs = flux_scale*image_plane_hs.sum(axis=(-1,-2)) + qu_sha

%matplotlib widget
axes = bhnerf.visualization.plot_stokes_lc(target, stokes, t_frames, label='True', plot_qu=True)
bhnerf.visualization.plot_stokes_lc(qu_hs, stokes, t_model, label='Gaussian', plot_qu=True, axes=axes, fmt='-', color='r')
for ax in axes: ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [159]:
"""
1. Rotate coordinates ato match Weilgus et al. (Fig. D.1.) 
2. Generate an ehtim Movie object (can be saved to mp4 using movie.export_mp4 function.
"""
movie = make_ehtim_movie(image_plane_hs, t_model, fov_M, stokes)
movie.get_frame(0).display('Q');


Merging 100 frames from MJD 57854 9.34 hr to MJD 57854 11.15 hr


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [167]:
%matplotlib widget
movies = [xr.DataArray(image_plane_hs[:,i], dims=['t','alpha','beta']) for i in range(len(stokes))]
fig, axes = plt.subplots(1,2,figsize=(8,3))
bhnerf.visualization.animate_movies_synced(movies, axes, titles=stokes)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
%matplotlib widget
axes = bhnerf.visualization.plot_stokes_lc(target, stokes, t_frames, label='True', plot_qu=True)
bhnerf.visualization.plot_stokes_lc(qu_semi_analytic, stokes, t_model, label='Point source', plot_qu=True, axes=axes, fmt='--', color='r')
bhnerf.visualization.plot_stokes_lc(qu_hs, stokes, t_model, label='Gaussian', plot_qu=True, axes=axes, fmt='-', color='r')
for ax in axes: ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …